In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,52,2024-02-10,-1,120.764706,98.733333,0.577176,10.594118,20.788235,0.207314,121.356863,...,10.628846,22.150000,0.202981,118.944231,0,2,5,4,21,2024
1,52,2024-02-10,2,113.294118,99.574510,0.539745,13.123529,25.156863,0.200275,113.145098,...,11.098000,25.144000,0.215800,121.940000,0,2,5,2,9,2024
2,53,2024-02-10,-6,112.153846,100.963462,0.530308,12.542308,22.482692,0.173308,110.761538,...,10.533333,25.345098,0.179451,116.247059,0,2,5,2,27,2024
3,53,2024-02-10,2,111.538462,95.525000,0.533942,10.809615,24.340385,0.189231,115.234615,...,12.365385,25.734615,0.229692,113.784615,0,2,5,2,4,2024
4,52,2024-02-10,-4,118.509804,98.109804,0.538157,9.986275,25.129412,0.253059,120.368627,...,11.725490,19.717647,0.178922,112.043137,0,2,5,1,23,2024
5,53,2024-02-10,-7,107.096154,98.136538,0.516231,12.475000,22.159615,0.182885,108.775000,...,11.668627,22.260784,0.171314,110.531373,0,2,5,2,15,2024
6,51,2024-02-10,8,114.820000,97.652000,0.558240,11.708000,24.788000,0.183660,117.148000,...,11.392308,25.065385,0.188058,115.684615,0,2,5,2,6,2024
7,54,2024-02-10,-1,124.037736,101.709434,0.582925,11.345283,24.109434,0.181660,121.966038,...,11.001923,29.380769,0.205962,119.753846,0,2,5,2,12,2024
8,52,2024-02-10,-3,113.509804,98.235294,0.531373,11.011765,23.937255,0.205294,114.433333,...,10.900000,27.842308,0.217731,119.840385,0,2,5,1,11,2024
9,53,2024-02-10,3,117.500000,97.557692,0.571615,12.607692,24.738462,0.236750,119.886538,...,12.081633,27.471429,0.197735,119.263265,0,2,5,2,24,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'FCNN':
        fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
        fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)
        
        model_file = fcnns.iloc[0]['File']
        
        model = torch.load('Models/'+model_file)
        scaler = joblib.load('Models/Scalers/FCNN.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'FCNN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-10,OKC,DAL,1.740741,2.140000,0
1,2024-02-10,DET,LAC,9.500000,1.068966,0
2,2024-02-10,SAS,BRK,2.800000,1.458716,0
3,2024-02-10,CHI,ORL,2.700000,1.487805,0
4,2024-02-10,PHI,WAS,1.571429,2.450000,1
5,2024-02-10,MEM,CHO,1.588235,2.420000,0
6,2024-02-10,CLE,TOR,1.289855,3.750000,1
7,2024-02-10,IND,NYK,1.520833,2.600000,0
8,2024-02-10,HOU,ATL,2.640000,1.505051,0
9,2024-02-10,PHO,GSW,1.769231,2.100000,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-10,OKC,DAL,1.740741,2.140000,0
1,2024-02-10,DET,LAC,9.500000,1.068966,0
2,2024-02-10,SAS,BRK,2.800000,1.458716,0
3,2024-02-10,CHI,ORL,2.700000,1.487805,1
4,2024-02-10,PHI,WAS,1.571429,2.450000,1
5,2024-02-10,MEM,CHO,1.588235,2.420000,0
6,2024-02-10,CLE,TOR,1.289855,3.750000,0
7,2024-02-10,IND,NYK,1.520833,2.600000,0
8,2024-02-10,HOU,ATL,2.640000,1.505051,0
9,2024-02-10,PHO,GSW,1.769231,2.100000,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-10,OKC,DAL,1.740741,2.140000,0
1,2024-02-10,DET,LAC,9.500000,1.068966,0
2,2024-02-10,SAS,BRK,2.800000,1.458716,0
3,2024-02-10,CHI,ORL,2.700000,1.487805,0
4,2024-02-10,PHI,WAS,1.571429,2.450000,0
5,2024-02-10,MEM,CHO,1.588235,2.420000,0
6,2024-02-10,CLE,TOR,1.289855,3.750000,0
7,2024-02-10,IND,NYK,1.520833,2.600000,1
8,2024-02-10,HOU,ATL,2.640000,1.505051,0
9,2024-02-10,PHO,GSW,1.769231,2.100000,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-10,OKC,DAL,1.740741,2.140000,0
1,2024-02-10,DET,LAC,9.500000,1.068966,0
2,2024-02-10,SAS,BRK,2.800000,1.458716,0
3,2024-02-10,CHI,ORL,2.700000,1.487805,1
4,2024-02-10,PHI,WAS,1.571429,2.450000,1
5,2024-02-10,MEM,CHO,1.588235,2.420000,0
6,2024-02-10,CLE,TOR,1.289855,3.750000,1
7,2024-02-10,IND,NYK,1.520833,2.600000,0
8,2024-02-10,HOU,ATL,2.640000,1.505051,0
9,2024-02-10,PHO,GSW,1.769231,2.100000,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-10,OKC,DAL,1.740741,2.140000,0.0
1,2024-02-10,DET,LAC,9.500000,1.068966,0.0
2,2024-02-10,SAS,BRK,2.800000,1.458716,0.0
3,2024-02-10,CHI,ORL,2.700000,1.487805,1.0
4,2024-02-10,PHI,WAS,1.571429,2.450000,1.0
5,2024-02-10,MEM,CHO,1.588235,2.420000,0.0
6,2024-02-10,CLE,TOR,1.289855,3.750000,0.0
7,2024-02-10,IND,NYK,1.520833,2.600000,0.0
8,2024-02-10,HOU,ATL,2.640000,1.505051,0.0
9,2024-02-10,PHO,GSW,1.769231,2.100000,0.0
